# Paquetes

In [ ]:
!pip install pdfplumber

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pdfplumber
import os
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Funciones

## Get Index

In [ ]:
from itertools import islice

def get_index(iterable, value, n):
    matches = (idx for idx, val in enumerate(iterable) if value in val)
    return next(islice(matches, n-1, n), None)

## Municipios

In [ ]:
def municipios(info=None, info2=None, info3=None):
    tipo = []
    titular = []
    cuit = []
    condicion = []
    programa = []
    capital_fijo = []
    capital_variable = []
    nosis = []
    fecha = []
    ingresos_mensuales = []
    cuota_promedio = []

    # Tipo
    tipo.append('Dictamen Tecnico Persona Juridica')

    # Titular
    titular1 = get_index(info, "TITULAR", 1)
    ntango = get_index(info, "N°", 1)

    titular_lista = []
    for i in range(titular1+3, ntango):
      titular_lista.append(info[i])

    titular.append(' '.join(titular_lista))

    # CUIT
    cuit1 = get_index(info, "CUIT", 1)
    cuit.append(info[cuit1+1])

    # Condición
    cond1 = get_index(info, "Cond.", 1)
    if info[cuit1-1] == "Resp.": 
      condicion.append("Resp. Inscripto")
    else:
      condicion.append(info[cond1+2])

    # Programa
    programa1 = get_index(info, 'ENCUADRE', 1)
    programa2 = get_index(info, 'Monto', 1)
    programa3 = get_index(info, 'Crédito', 1)
    programa4 = get_index(info, 'Programa', 1)
    programa5 = get_index(info, '%', 1)
    programa6 = get_index(info, '%', 3)

    prog_lista = []

    for i in range(programa1+5, programa2):
      prog_lista.append(info[i])

    for i in range(programa3+3, programa4):
      prog_lista.append(info[i])

    for i in range(programa4+1, programa5):
      prog_lista.append(info[i])

    for i in range(programa6-2, programa6):
      prog_lista.append(info[i])

    programa.append(' '.join(prog_lista))

    # Capital Fijo
    capfijo1 = get_index(info, 'Capital', 1)
    capital_fijo.append(info[capfijo1+2])

    # Capital Variable
    capvar1 = get_index(info, 'Capital', 2)
    capital_variable.append(info[capvar1+3])

    # Score Nosis
    score1 = get_index(info, 'SCORE', 1)
    nosis.append(info[score1+1])

    # Fecha
    fecha.append(info[-1])

    # Ingresos Mensuales
    ingresos1 = get_index(info, 'Total', 1)
    ingresos_mensuales.append(info[ingresos1+4])

    # Cuota Promedio
    cuota1 = get_index(info, 'Promedio', 1)
    cuota_promedio.append(info[cuota1+2])

    return tipo, titular, cuit, condicion, programa, capital_fijo, capital_variable, nosis, fecha, ingresos_mensuales, cuota_promedio

## Particulares

In [ ]:
def particulares(info=None, info2=None, info3=None):
    titular = []
    cuit = []
    condicion = []
    programa = []
    capital_fijo = []
    capital_variable = []
    nosis = []
    fecha = []
    ingresos_mensuales = []
    cuota_promedio = []
    tipo = []

    # Tipo
    tipo.append('Dictamen Tecnico Persona Física')

    # Titular
    titular1 = get_index(info, "TITULAR", 1)
    atm = get_index(info, "ATM", 1)

    titular_lista = []
    for i in range(titular1+3, atm-1):
      titular_lista.append(info[i])

    titular.append(' '.join(titular_lista))

    # CUIT
    cuit1 = get_index(info, "CUIT", 1)
    cuit.append(info[cuit1+1])

    # Condición
    cond1 = get_index(info, "AFIP", 1)

    if info[cond1+1] == "Inscripto" or "Resp.": 
      condicion.append("Resp. Inscripto")
    else:
      condicion.append(info[cond1+1])

    # Programa
    find_value = False
    try:
      programa1 = get_index(info, 'DESTINO', 1)
      programa2 = get_index(info, 'Programa', 1)
      programa3 = get_index(info, 'crédito', 1)
      programa4 = get_index(info, 'Línea', 1)

      programa5 = []

      for i in range(programa1+4, programa2-1):
        programa5.append(info[i])

      for i in range(programa3+3, programa4):
        programa5.append(info[i])

      programa5 = (' '.join(programa5))
      if len(programa5) > 0:
        programa.append(programa5)
        find_value = True
      else:
        pass
    except:
      pass
    
    if find_value == False:
      try:
        programa1 = get_index(info, 'Programa', 1)
        programa2 = get_index(info, 'Monto', 1)

        programa3 = []

        for i in range(programa1+1, programa2):
          programa3.append(info[i])

        programa3 = (' '.join(programa3))
        if len(programa3) > 0:
          programa.append(programa3)
          find_value = True
        else:
          pass
      except:
        pass

    if find_value == False:
      programa1 = get_index(info, 'Programa', 1)

      try:
        if str(info[programa1+1]).upper() == 'MICROCRÉDITO':
          programa.append(info[programa1+1])
        else:
          programa.append(np.nan)
      except:
        pass
    else:
      pass

    # Capital Fijo
    try:
      capfijo1 = get_index(info, 'Activo', 1)
      capital_fijo.append(info[capfijo1+3])
    except:
      capital_fijo.append(np.nan)

    # Capital Variable
    try:
      capvar1 = get_index(info, 'Capital', 1)
      if '%' in info[capvar1+4]:
        capital_variable.append(info[capvar1+4])
      else:
        capvar1 = get_index(info, 'Capital', 2)
        capital_variable.append(info[capvar1+4])
    except:
      capital_variable.append(np.nan)

    # Score Nosis
    try:
      score1 = get_index(info2, 'SCORE', 1)
      nosis.append(info2[score1+1])
    except:
      score1 = get_index(info, 'SCORE', 1)
      nosis.append(info[score1+1])

    # Fecha
    fecha.append(info2[-1])

    # Ingresos Mensuales
    try:
      ingresos1 = get_index(info2, 'TOTALES', 1)
      ingresos_mensuales.append(info2[ingresos1+2])
    except:
      ingresos1 = get_index(info, 'TOTALES', 1)
      ingresos_mensuales.append(info[ingresos1+2])

    # Cuota Promedio
    try:
      cuota1 = get_index(info2, 'Cuota', 2)
      cuota_promedio.append(info2[cuota1+3])
    except:
      cuota1 = get_index(info, 'Cuota', 2)
      cuota_promedio.append(info[cuota1+3])

    return tipo, titular, cuit, condicion, programa, capital_fijo, capital_variable, nosis, fecha, ingresos_mensuales, cuota_promedio

## Preincubación

In [ ]:
def preincubacion(info=None, info2=None, info3=None):
    titular = []
    cuit = []
    condicion = []
    programa = []
    capital_fijo = []
    capital_variable = []
    nosis = []
    fecha = []
    ingresos_mensuales = []
    cuota_promedio = []
    tipo = []

    # Tipo
    tipo.append('Pre-Incubacion')

    # Titular
    titular_lista = []
    titular1 = get_index(info, "TITULAR", 1)
    atm = get_index(info, "ATM", 1)
    for i in range(titular1+3, atm-1):
      titular_lista.append(info[i])

    titular.append(' '.join(titular_lista))

    # CUIT
    cuit1 = get_index(info, "CUIT", 1)
    cuit.append(info[cuit1+1])

    # Condición
    cond1 = get_index(info, "AFIP", 1)
    if info[cond1+1] == "Inscripto": 
      condicion.append("Resp. Inscripto")
    else:
      condicion.append(info[cond1+1])

    # Programa
    # programa1 = get_index(info, 'Programa', 1)
    # programa.append(info[programa1+1])
    programa.append('Preincubacion')

    # Capital Fijo
    capfijo1 = get_index(info, 'Activo', 1)
    capital_fijo.append(info[capfijo1+3])

    # Capital Variable
    capvar1 = get_index(info, 'Capital', 1)
    if '%' in info[capvar1+4]:
      capital_variable.append(info[capvar1+4])
    else:
      capvar1 = get_index(info, 'Capital', 2)
      capital_variable.append(info[capvar1+4])

    # Score Nosis
    try:
      score1 = get_index(info2, 'SCORE', 1)
      nosis.append(info2[score1+1])
    except:
      score1 = get_index(info, 'SCORE', 1)
      nosis.append(info[score1+1])

    # Fecha
    fecha.append(info2[-1])

    # Ingresos Mensuales
    try:
      ingresos1 = get_index(info2, 'TOTALES', 1)
      ingresos_mensuales.append(info2[ingresos1+2])
    except:
      ingresos1 = get_index(info, 'TOTALES', 1)
      ingresos_mensuales.append(info[ingresos1+2])

    # Cuota Promedio
    try:
      cuota1 = get_index(info2, 'Cuota', 2)
      cuota_promedio.append(info2[cuota1+3])
    except:
      cuota1 = get_index(info, 'Cuota', 2)
      cuota_promedio.append(info[cuota1+3])
    
    return tipo, titular, cuit, condicion, programa, capital_fijo, capital_variable, nosis, fecha, ingresos_mensuales, cuota_promedio

## Recrédito

In [ ]:
def recredito(info=None, info2=None, info3=None): 
    titular = []
    cuit = []
    condicion = []
    programa = []
    capital_fijo = []
    capital_variable = []
    nosis = []
    fecha = []
    ingresos_mensuales = []
    cuota_promedio = []
    tipo = []

    # Tipo
    tipo.append('Re-Crédito')

    # Titular
    titular1 = get_index(info, "PROYECTO", 1)
    titular2 = get_index(info, "Nro", 1)

    titular_lista = []
    for i in range(titular1+1, titular2):
      titular_lista.append(info[i])

    titular.append(' '.join(titular_lista))

    # CUIT
    cuit1 = get_index(info, "CUIT", 1)
    cuit.append(info[cuit1+1])

    # Condición
    cond1 = get_index(info, "AFIP", 1)

    if info[cond1+1] == "Inscripto" or "Resp.": 
      condicion.append("Resp. Inscripto")
    else:
      condicion.append(info[cond1+1])

    # Programa
    programa1 = get_index(info, 'Programa', 1)
    programa.append('Re-Crédito')

    # Capital Fijo
    capfijo1 = get_index(info, 'Activo', 1)
    capital_fijo.append(info[capfijo1+2])

    # Capital Variable
    capvar1 = get_index(info, 'Capital', 1)
    if '%' in info[capvar1+6]:
      capital_variable.append(info[capvar1+6])
    else:
      capital_variable.append(info[capvar1+3])

    # Score Nosis
    found_info = False
    try:
      score1 = get_index(info2, 'SCORE', 1)
      nosis.append(info2[score1+1])
      found_info = True
    except:
      pass

    if not found_info:
      try:
        score1 = get_index(info3, 'SCORE', 1)
        nosis.append(info3[score1+1])
      except:
        score1 = get_index(info, 'SCORE', 1)
        nosis.append(info[score1+1])

    # Fecha
    fecha.append(info2[-1])

    # Ingresos Mensuales
    found_info = False
    try:
      ingresos1 = get_index(info2, 'SCORE', 1)
      ingresos_mensuales.append(info2[ingresos1-2])
      found_info = True
    except:
      pass

    if not found_info:
      try:
        ingresos1 = get_index(info3, 'SCORE', 1)
        ingresos_mensuales.append(info3[ingresos1-2])
      except:
        ingresos1 = get_index(info, 'SCORE', 1)
        ingresos_mensuales.append(info[ingresos1-2])

    # Cuota Promedio
    found_info = False
    try:
      cuota1 = get_index(info2, 'Cuota', 2)
      cuota_promedio.append(info2[cuota1+3])
      found_info = True
    except:
      pass
    
    if not found_info:
      try:
        cuota1 = get_index(info3, 'Cuota', 2)
        cuota_promedio.append(info3[cuota1+3])
      except:
        cuota1 = get_index(info, 'Cuota', 2)
        cuota_promedio.append(info[cuota1+3])
    
    return tipo, titular, cuit, condicion, programa, capital_fijo, capital_variable, nosis, fecha, ingresos_mensuales, cuota_promedio

## Extracción

In [ ]:
def extraer(pdf_path):
  titular = []
  cuit = []
  condicion = []
  programa = []
  capital_fijo = []
  capital_variable = []
  nosis = []
  fecha = []
  ingresos_mensuales = []
  cuota_promedio = []
  tipo = []

  files = [f for f in os.listdir(pdf_path) if f.endswith(".pdf")]
  path = pdf_path

  for file in files:
    filepath = str(pdf_path)+file
    with pdfplumber.open(filepath) as pdf:
      first_page = pdf.pages[0]
      page_text = first_page.extract_text()

      page_text1 = page_text.replace('\n',' ').split(' ')
      info = []
      for i in page_text1:
        if i != '':
          info.append(i)

      try:
        second_page = pdf.pages[1]
        page_text = second_page.extract_text()

        page_text2 = page_text.replace('\n',' ').split(' ')
        info2 = []
        for i in page_text2:
          if i != '':
            info2.append(i)
      except:
        pass

      try:
        third_page = pdf.pages[2]
        page_text = third_page.extract_text()

        page_text3 = page_text.replace('\n',' ').split(' ')
        info3 = []
        for i in page_text3:
          if i != '':
            info3.append(i)
      except:
        pass
      
      if info[1] == 'PRE-INCUBACIÓN':
        tipo_temp, titular_temp, cuit_temp, condicion_temp, programa_temp, capital_fijo_temp, capital_variable_temp, nosis_temp, fecha_temp, ingresos_mensuales_temp, cuota_promedio_temp = preincubacion(info, info2)
      elif info[2] == 'RE-CRÉDITO':
        tipo_temp, titular_temp, cuit_temp, condicion_temp, programa_temp, capital_fijo_temp, capital_variable_temp, nosis_temp, fecha_temp, ingresos_mensuales_temp, cuota_promedio_temp = recredito(info, info2, info3)
      elif info[3] == 'MUNICIPIO':
        tipo_temp, titular_temp, cuit_temp, condicion_temp, programa_temp, capital_fijo_temp, capital_variable_temp, nosis_temp, fecha_temp, ingresos_mensuales_temp, cuota_promedio_temp = municipios(info)
      else:
        tipo_temp, titular_temp, cuit_temp, condicion_temp, programa_temp, capital_fijo_temp, capital_variable_temp, nosis_temp, fecha_temp, ingresos_mensuales_temp, cuota_promedio_temp = particulares(info, info2)

    tipo.extend(tipo_temp)
    titular.extend(titular_temp)
    cuit.extend(cuit_temp)
    condicion.extend(condicion_temp)
    programa.extend(programa_temp)
    capital_fijo.extend(capital_fijo_temp)
    capital_variable.extend(capital_variable_temp)
    nosis.extend(nosis_temp)
    fecha.extend(fecha_temp)
    ingresos_mensuales.extend(ingresos_mensuales_temp)
    cuota_promedio.extend(cuota_promedio_temp)

  df = pd.DataFrame({'Fecha':fecha,
                     'Tipo':[i.upper() for i in tipo],
                    'Titular':[i.upper() for i in titular],
                    'CUIT':cuit,
                    'Condicion':[i.upper() for i in condicion],
                    'Programa':[i.upper() for i in programa],
                    'Capital Fijo':capital_fijo,
                    'Capital Variable':capital_variable,
                    'Score Nosis':nosis,
                    'Ingresos Mensuales':ingresos_mensuales,
                    'Cuota Promedio':cuota_promedio
                      })

  return df

# Ejecución y guardado

In [ ]:
df = extraer('/content/drive/MyDrive/Fondo Credito/')

In [ ]:
df['Condicion'] = df['Condicion'].replace('NO','NO INSCRIPTO')
df

In [ ]:
df.to_excel('/content/drive/MyDrive/Fondo Credito/Output.xlsx', index=False)